### Cargar el archivo

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD 
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words_sp = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# A la base de comentarios de Tiktok se le agrega:
    # - Una lista de palabras positivos
    # - Una lista de palabra negativas

In [52]:
df = pd.read_excel('Tiktok-Prueba2.xlsx')
df.head()

,Unnamed: 0,name_video,date_video,date_comment,detail_comment,like_comment,positivo,negativo,neutro,sent
0,1500.0,¡Ley aprobada para reducir el precio de los al...,2022-08-04 00:00:00,2022-08-04 00:00:00,"Marícarmen Alva ,quienes han aportado en la ON...",1.0,0,0,1,neutro
1,1501.0,¡Ley aprobada para reducir el precio de los al...,2022-08-04 00:00:00,2013-04-01 00:00:00,Señora presidenta del congreso podrían hacer a...,0.0,0,0,1,neutro
2,1502.0,¡Ley aprobada para reducir el precio de los al...,2022-08-04 00:00:00,2022-12-04 00:00:00,86 %de desaprobación,0.0,0,1,0,negativo
3,1503.0,¡Ley aprobada para reducir el precio de los al...,2022-08-04 00:00:00,2022-11-04 00:00:00,recuerde que Dios todo poderoso la mira del ci...,0.0,0,0,1,neutro
4,1504.0,¡Ley aprobada para reducir el precio de los al...,2022-08-04 00:00:00,2022-11-04 00:00:00,que sus promesas se hagan realidad por el pueblo,0.0,0,0,1,neutro


In [53]:
df.sent.value_counts()

negativo    1200
neutro       637
positivo     362
Name: sent, dtype: int64

### Limpieza del texto (puntuación, tokenización y eliminado de stopwords en español)

In [54]:
from nltk.stem import SnowballStemmer
stm = SnowballStemmer('spanish') 
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stop_words_sp]
    stemi = [stm.stem(token) for token in text]
    return stemi

"""
stemmer = SnowballStemmer("spanish")
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stop_words_sp]
    #stemi = [stemmer.stem(token) for token in text]
    complete = " ".join(text)
    return complete
"""

'\nstemmer = SnowballStemmer("spanish")\ndef clean_text(text):\n    text = "".join([word.lower() for word in text if word not in string.punctuation])\n    tokens = re.split(\'\\W+\', text)\n    text = [word for word in tokens if word not in stop_words_sp]\n    #stemi = [stemmer.stem(token) for token in text]\n    complete = " ".join(text)\n    return complete\n'

### Matriz Documento-Término (DTM) con puntuaciones TF-IDF

In [55]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(df['detail_comment'])

In [56]:
X_features = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vect.get_feature_names())
X_features.head()

C:\Program Files (x86)\ANACONDA\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,,04,1,10,100,101,112,115,12,130,...,zopenc,çierr,ñiñ,ďia,𝚌𝚒𝚎𝚛𝚛𝚎,𝚌𝚘𝚗𝚐𝚛𝚎𝚜𝚘,𝚌𝚘𝚗𝚜𝚝𝚒𝚝𝚞𝚌𝚒o𝚗,𝚍𝚎𝚕,𝚗𝚞𝚎𝚟𝚊,𝚢
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
X_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2199 entries, 0 to 2198
Columns: 2522 entries,  to 𝚢
dtypes: float64(2522)
memory usage: 42.3 MB


### Removemos las columnas de términos redundantes y los que son poco frecuentes

In [67]:
column = pd.DataFrame(X_features.sum())
column.head()

,0
04,0.571170
1,0.575815
10,0.979539
100,0.577945
101,0.625018


In [59]:
X_features.drop(columns=['','27','28'], inplace=True)

In [60]:
X_features.drop(columns=['4','5'], inplace=True)

In [69]:
stop = list(column[column[0]<3].index)

In [70]:
len(stop)

2237

In [71]:
X_features.drop(columns=stop, inplace=True)

In [72]:
X_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2199 entries, 0 to 2198
Columns: 280 entries, abuelit to ya
dtypes: float64(280)
memory usage: 4.7 MB


In [73]:
X_features.head()

,abuelit,accion,adel,adelant,ahi,ahor,ala,alva,amig,anticuch,...,verd,verdader,vez,vide,visit,viv,vividor,vot,x,ya
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.209172,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
print([i for i in X_features.columns])

['abuelit', 'accion', 'adel', 'adelant', 'ahi', 'ahor', 'ala', 'alva', 'amig', 'anticuch', 'apoy', 'armad', 'asamble', 'asi', 'audi', 'años', 'bast', 'belaund', 'bendicion', 'bien', 'blind', 'bocon', 'bot', 'buen', 'burr', 'busc', 'caer', 'caeras', 'call', 'calla', 'cambi', 'cans', 'car', 'caraj', 'carm', 'cas', 'castill', 'cerr', 'charlat', 'chiabr', 'chist', 'cierr', 'ciert', 'clar', 'com', 'compr', 'comun', 'congres', 'constitu', 'constitucion', 'constituyent', 'corrupcion', 'corrupt', 'cos', 'cre', 'crisis', 'cuant', 'cuent', 'cuestion', 'culp', 'culpabl', 'da', 'dañ', 'deb', 'dec', 'defend', 'defiend', 'dej', 'democraci', 'derech', 'derrot', 'desaprob', 'desesper', 'despues', 'dia', 'dic', 'dich', 'dig', 'dij', 'dios', 'dirigent', 'disolv', 'eh', 'ejecut', 'eleccion', 'empres', 'engañ', 'entrar', 'escuch', 'esper', 'exact', 'excelent', 'falt', 'famili', 'favor', 'felic', 'felicit', 'feliz', 'fuerz', 'fujimori', 'futur', 'gan', 'general', 'gent', 'glori', 'gobiern', 'golp', 'golpis

#### Aplicamos la descomposición SVD para poder trabajar con menos variables (30)

In [75]:
truncatedSVD=TruncatedSVD(30)
X_svd = truncatedSVD.fit_transform(X_features)
X_svd[:10] 

array([[ 8.36564064e-03, -6.49089591e-04,  9.83051681e-03,
         2.90576408e-03, -4.65335408e-03,  4.23811937e-03,
         4.77413392e-03,  8.17491777e-03,  7.01268037e-03,
         1.89133134e-03,  4.23605527e-03,  7.63001713e-03,
         8.35204574e-03, -7.80227901e-03,  8.98994029e-03,
        -2.92951206e-04, -1.00201359e-02, -6.85759424e-04,
        -1.63613366e-02, -4.33490238e-03,  1.22232381e-03,
        -7.05612251e-03,  3.10736147e-03,  6.93039855e-03,
         5.49499282e-03, -1.41178094e-02,  7.13224819e-02,
         1.72127214e-02,  1.11635888e-02, -1.60273099e-02],
       [ 1.74261153e-01, -6.47989979e-02,  1.53448383e-01,
        -4.08924795e-02, -1.52432773e-02, -2.11638041e-02,
         1.17517215e-02,  4.99703329e-02,  6.32000242e-02,
        -1.30602180e-02, -2.80509235e-02, -7.33464276e-03,
        -4.55999968e-02,  5.18854322e-03, -4.65865961e-02,
         3.23215751e-02, -3.86418410e-02, -4.52495982e-02,
         2.09384609e-02,  2.22901867e-03, -1.30005152e-

In [76]:
len(X_svd)

2199

#### Creamos un Clasificador de Comentarios utilizando RandomForest

In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

Particionamos las muestras

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X_svd,
                                                    df['sent'],
                                                    test_size=0.2)

In [80]:
#print(X_train[:10])

print(y_train[:10])

339       neutro
247     negativo
1279    negativo
535     negativo
302     positivo
15      negativo
922       neutro
2085    negativo
1722      neutro
424       neutro
Name: sent, dtype: object


Entrenamos el modelo y generamos las predicciones sobre la muestra de evaluación

In [90]:
rf = RandomForestClassifier()
rf_model = rf.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

#### Métricas de Diagnóstico

In [91]:
print('Matriz de Confusión:')
confusion_matrix(y_test, y_pred)

Matriz de Confusión:


array([[206,  40,  15],
       [ 73,  30,  12],
       [ 22,   7,  35]], dtype=int64)

In [92]:
print('Accuracy: %.2f%%' %(accuracy_score(y_test, y_pred)*100))  
print('Precision: %.2f%%' % (precision_score(y_test, y_pred, average= 'weighted')*100))
print('Recall: %.2f%%' % (recall_score(y_test, y_pred, average= 'weighted')*100))

Accuracy: 61.59%
Precision: 58.99%
Recall: 61.59%


### LSTM
https://www.nbshare.io/notebook/249468051/How-To-Code-RNN-and-LSTM-Neural-Networks-in-Python/

In [42]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.__version__

'2.9.1'

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X_svd,
                                                    df[['positivo','negativo','neutro']],
                                                    test_size=0.2)

In [98]:
model = tf.keras.Sequential([
    #encoder, # the encoder
    tf.keras.layers.Embedding(
        input_dim=30,
        output_dim=1,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(layers.LSTM(64)), # making LSTM bidirectional
    tf.keras.layers.Dense(32, activation='relu'), # FC layer for the classification part
    tf.keras.layers.Dense(1) # final FC layer

])

In [48]:
model = tf.keras.Sequential([ tf.keras.layers.Embedding(input_dim=30, output_dim=3)])
model.add(tf.keras.layers.Bidirectional(layers.LSTM(64))) 
model.add(tf.keras.layers.Dense(32, activation='relu')) 
model.add(tf.keras.layers.Dense(32, activation='relu')) 
model.add(tf.keras.layers.Dense(32, activation='relu')) 
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='relu')) 
model.add(tf.keras.layers.Dense(3, activation='softmax')) 

In [49]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    # adam optimizer is more efficient (not always the most accurate though)
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 3)           90        
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              34816     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 32)                4128      
                                                                 
 dense_3 (Dense)             (None, 32)                1056      
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 16)                528       
                                                      

In [51]:
model.fit(X_train, y_train, epochs=20) 

Epoch 1/20


C:\Program Files (x86)\ANACONDA\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


55/55 [==============================] - 4s 15ms/step - loss: 0.6898 - accuracy: 0.5458
Epoch 2/20
55/55 [==============================] - 1s 17ms/step - loss: 0.6740 - accuracy: 0.5458
Epoch 3/20
55/55 [==============================] - 1s 15ms/step - loss: 0.6430 - accuracy: 0.5458
Epoch 4/20
55/55 [==============================] - 1s 16ms/step - loss: 0.6304 - accuracy: 0.5458
Epoch 5/20
55/55 [==============================] - 1s 15ms/step - loss: 0.6195 - accuracy: 0.5458
Epoch 6/20
55/55 [==============================] - 1s 15ms/step - loss: 0.6090 - accuracy: 0.5458
Epoch 7/20
55/55 [==============================] - 1s 15ms/step - loss: 0.5987 - accuracy: 0.5458
Epoch 8/20
55/55 [==============================] - 1s 15ms/step - loss: 0.5892 - accuracy: 0.5458
Epoch 9/20
55/55 [==============================] - 1s 15ms/step - loss: 0.5841 - accuracy: 0.5458
Epoch 10/20
55/55 [==============================] - 1s 15ms/step - loss: 0.5830 - accuracy: 0.5458
Epoch 11/20
55/55 [=